Exhausting a coroutine.

Coroutines use `async def` to be defined

In [1]:
import time

In [3]:
# simplest coroutine

async def simplest_coroutine():
    return "Yup, I am done."


simplest_coro_obj = simplest_coroutine()

print(simplest_coro_obj.send(None))  # first .send to a coroutine must always be `None` 


# When the coroutine function reaches the end it raises a StopIteration
# The StopIteration obj contains a `value` property with the function's return, if any, if not None.
 

StopIteration: Yup, I am done.

In [4]:
# The caller function (the one calling the .send) is the one that should handle the StopIteration error.
# Something like..

simplest_coro_obj2 = simplest_coroutine()

try:

    simplest_coro_obj2.send(None)

except StopIteration as e:
    print("Return value: ", e.value)



Return value:  Yup, I am done.


In [5]:
# More complex coroutines


from collections.abc import Awaitable
class MyIterator(Awaitable):
    def __init__(self, iterator):
        self.iterator = iterator

    def __iter__(self):
        return self.iterator

    def __next__(self):
        return next(self.iterator)

    __await__ = __iter__


In [6]:
async def coroutine1():
    sequence = [1,2,3,4,5,6]

    await MyIterator(iter(sequence)) 
    # await expects an interator but a plain one wont do. Bc `await` expects an object that implements `__await__`
    # this `__await__` function should return an iterator (a generator iterator would do as well)

    return "coroutine1: Return after await"

print(type(coroutine1))
print(type(coroutine1()))

<class 'function'>
<class 'coroutine'>


  print(type(coroutine1()))


In [ ]:
coro1_obj = coroutine1()

print(coro1_obj.send(None))  #1
print(coro1_obj.send(None))  #2
print(coro1_obj.send(None))  #3
print(coro1_obj.send(None))  #4
print(coro1_obj.send(None))  #5
print(coro1_obj.send(None))  #6

print(coro1_obj.send(None))  # StopIteration error from MyIterator

# the coroutine in itself acts as an iterator, so when exhausted the coroutine itself raises StopIteration with the return value 


1
2
3
4
5
6


StopIteration: coroutine1: Return after await

In [8]:
# another sample with a dedicated awaitable

from typing import Any, Generator


class MyAwaitable(Awaitable):  # MyAwaitable is not made iterable (implement __iter__) on purpose. it is o
    def __await__(self) -> Generator[Any, Any, Any]:
        yield
        yield 1
        yield 2
        yield 3

        print("No more iteration after this point (it will raise: StopIteration)")

        return "This awaitable has been exhausted, Bye."  # like raise: StopIteration(value=<return_value>)
    

# Run the generator defined in the awaitbale using a for .. in    (hint: no StopIteration error)

my_awaitable = MyAwaitable()

for a in my_awaitable.__await__():
    print("output: ", a)


# Notice: The return value is not treated as an iteration value. (No "output: ")
# The return value is not meant to be one.

output:  None
output:  1
output:  2
output:  3
No more iteration after this point (it will raise: StopIteration)


In [9]:
# Run the generator defined in the awaitbale by hand/manually..   (Hint: StopIteration error is raised)
# In the above example the for .. in .. handles the StopIteration for you

gen = MyAwaitable().__await__()

print("Output:", gen.send(None))
print("Output:", gen.send(None))
print("Output:", gen.send(None))
print("Output:", gen.send(None))

print("Output:", gen.send(None))


Output: None
Output: 1
Output: 2
Output: 3
No more iteration after this point (it will raise: StopIteration)


StopIteration: This awaitable has been exhausted, Bye.

In [10]:
# Just catch the error..

gen = MyAwaitable().__await__()

print("Output:", gen.send(None))
print("Output:", gen.send(None))
print("Output:", gen.send(None))
print("Output:", gen.send(None))

try:
    print("Output:", gen.send(None))
except StopIteration as e:
    print('Return:', e.value)

Output: None
Output: 1
Output: 2
Output: 3
No more iteration after this point (it will raise: StopIteration)
Return: This awaitable has been exhausted, Bye.


In [ ]:
# Awaitable and native coroutines


async def coroutine2():  # native coroutines are not iterators (no -> next(nativecoroutine_obj))
    output = await MyAwaitable()  
    # the intermediate values yielded by `my_awaitable` are not received by coroutine2
    # but by whatever calling function is exhausting it  (calling its .send(..) function )
    print("msg from coroutine2: ", output)
    return output


coro2_obj = coroutine2()

# exhausting the coroutine obj  (use send and always pass None or a value to it)
print(coro2_obj.send(None))  # None, as the first yield
print(coro2_obj.send(None))  # 1
print(coro2_obj.send(None))  # 2
print(coro2_obj.send(None))  # 3

time.sleep(3)
print(coro2_obj.send(None))  # raises StopIteration


None
1
2
3
No more iteration after this point (it will raise: StopIteration)
msg from coroutine2:  This awaitable has been exhausted, Bye.


StopIteration: This awaitable has been exhausted, Bye.

In [12]:
# Handling the errors above would be like..


async def coroutine2():
    output = await MyAwaitable()  
    print("coroutine2 received output from my_awaytable: ", output)
    time.sleep(3)
    print("coroutine 2 is about to finish")
    time.sleep(3)
    
    return "coroutine2 return value: " + output
try:  # same code as before below

    coro2_obj = coroutine2()

    print(coro2_obj.send(None))  # None, as the first yield
    print(coro2_obj.send(None))  # 1
    print(coro2_obj.send(None))  # 2
    print(coro2_obj.send(None))  # 3

    print(coro2_obj.send(None))  # raises StopIteration

except StopIteration as e:
    print("StopIteration being handled. Return value: ", e.value)


# The StopIteration that needs handling is the one raised from the native coroutine.
# the await will handle the one raised by the awaitable 



None
1
2
3
No more iteration after this point (it will raise: StopIteration)
coroutine2 received output from my_awaytable:  This awaitable has been exhausted, Bye.
coroutine 2 is about to finish
StopIteration being handled. Return value:  coroutine2 return value: This awaitable has been exhausted, Bye.


In [ ]:
# Use a loop
# since you do not know or control how many items the coroutine's generator contains
# You cannot ask for len
# The native coroutine is not an iterator so you can't for..in..
# Then use..


coro2_obj1 = coroutine2()
while True:
    try:
        print("Output:", coro2_obj1.send(None))
    except StopIteration as e:
        print("Return:", e.value)
        break
     

Output: None
Output: 1
Output: 2
Output: 3
No more iteration after this point (it will raise: StopIteration)
coroutine2 received output from my_awaytable:  This awaitable has been exhausted, Bye.
coroutine 2 is about to finish
Return: coroutine2 return value: This awaitable has been exhausted, Bye.


So, in summary:

- `await` will handle the StoopIteration raised from the awaitable
- When you manually exhaust a coroutine (`.send`), and that coroutine ends it will raise StopIteration
- Use try.. except to handle the StopIteration error
- To get the coroutine return value use the StopIteration's property`value`. 
